In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Qui vengono importate le librerie necessarie

In [2]:
# statistical results
from statsmodels.formula.api  import ols
import statsmodels.api as sm


# visualization
import matplotlib.pyplot as plt
import pandas as pd
#%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree 
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn import svm
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

# outliers detection
from sklearn.ensemble import IsolationForest

Lo scopo di questo esercizio consiste nel combinare e mediare le previsioni sul dataset di test, partendo da comuni modelli di classificazione e regressione addestrati sul train dataset. Gli algoritmi utilizzati sono: il Support Vector Machine, il Gaussian Naive Bayes, il Decision Tree, la Random Forest, il K-Nearest Neighbors, e la Regressione Logistica. 

Per realizzare un classificatore Ensemble in grado di produrre un vettore di previsioni per il dataset di test, è necessario stimare vari modelli appartenenti alle famiglie selezionate, e selezionare quei modelli che presentano una migliore accuratezza in fase di convalida. Per fare questo è stata condotta un'ottimizzazione dei parametri più influenti. è stato anche selezionato il migliore modello per ogni tipo di classificatore

Infine, verrà sottoposto a Kaggle un vettore di previsioni (418\ \ast\ 1), i cui risultati saranno i valori medi di ogni test datapoint ottenuti considerando tutti i classificatori selezionati, e attribuendo un peso maggiore ai classificatori con la migliore accuracy in fase di convalida. Si utilizza infatti un approccio ensemble perché si tiene conto di vari classificatori.

In [3]:
#train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
#test_df = pd.read_csv('/kaggle/input/titanic/test.csv')
#train_df = pd.read_csv('C:/Users/Tommaso/Documents/Python Scripts\MACHINE LEARNING with UDACITY/Less2 p19 classification cars/titanic/train.csv')
train_df = pd.read_csv('Barbiero Tommaso Progetto SuperGuida Machine Learning e AI/train.csv')
#test_df = pd.read_csv('C:/Users/Tommaso/Documents/Python Scripts\MACHINE LEARNING with UDACITY/Less2 p19 classification cars/titanic/test.csv')
test_df = pd.read_csv('Barbiero Tommaso Progetto SuperGuida Machine Learning e AI/test.csv')

FileNotFoundError: [Errno 2] File Barbiero Tommaso Progetto SuperGuida Machine Learning e AI/train.csv does not exist: 'Barbiero Tommaso Progetto SuperGuida Machine Learning e AI/train.csv'

Combino i set di dati di train e di test in una sola, in modo da poter gestire contemporaneamente le features di entrambi i set di dati

In [ ]:
train_df.describe()

In [ ]:
#distance Ny Southampton 5950
#distance Ny Cherbourg 5800
#distance Ny Cogh 5210

test_df["Survived"]=np.empty(test_df.shape[0])
test_df["Survived"] = np.nan

test_df

La riga seguente rende omogeneo l'ordine dei dataset di train e di test

In [ ]:
train_df=train_df[["PassengerId","Pclass","Name","Sex","Age","SibSp","Parch","Ticket","Fare","Cabin","Embarked","Survived"]]

Combino i set di dati di train e di test in una sola, in modo da poter gestire contemporaneamente le features di entrambi i set di dati

In [ ]:
all_df=[train_df,test_df]

#IT: Pulizia del codice:
    -con la riga dataset['Age'][np.isnan(dataset['Age'])] = age_avg si assegna il valore medio di età ai datapoints aventi valore "nan"
    -si rende quantitativa la variabile 'Embarked' trasformandola nella distanza da New York (destinazione designata del Titanic)(Southhampton-NY: 5910 km, Cherbourg-NY: 5800 km, Queenstown: 5210 km). Si presume che il prezzo pagato dipenda anche dal porto di partenza. Quindi viene creata la variabile quantitativa 'Distance to NY'
    - Ai valori "0" o "nan" di "Fare" il prezzo del biglietto viene assegnato il valore medio del prezzo del biglietto con cui la persona ha viaggiato
    - Si crea una nuova variabile "Familiars" che somma  le variabili 'SibSp' (relativa a Coniugi e Cognati) e 'Parch' (relativa al numero di genitori e figli)
    - si sposta la colonna 'Survived' per avere un'idea più distinta sulle features e la label utilizzata 
    - due datapoints del train_set (indici 61 e 829) hanno un nan value sulla colonna 'Embarked' (e conseguentemente lo avrebbero sulla colonna "Distance to NY". Sono stati imputati questi valori a "Southhampton" sia perché è il porto da cui sono partiti la maggioranza dei viaggiatori, sia perché diversi viaggiatori che hanno pagato una tariffa molto simile ai 80 pounds di queste due persone provenivano da Southhampton. D'altronde i viaggiatori saliti a Queenstown che sono più vicini agli 80 pounds sono per difetto 29.125 sterline e per eccesso 90 sterline. Quindi è poco probabile che questi viaggiatori fossero saliti dal porto Irlandese (nettamente più vicini a New York di Southhampton e Cherbourg, questi ultimi due porti sono tra loro vicini)

In [ ]:
# Dato che all_df è una lista composta da due datasets (train e test), tutte le operazioni di pre-processamento si riferiscono a entrambi
for dataset in all_df:
    
    age_avg = dataset['Age'].mean()
    dataset['Fare']=dataset['Fare'].replace(0,-1)
    #assegno -1 ai valori nulli della colonna dataset['Fare']
    dataset['Fare'][np.isnan(dataset['Fare'])] = -1
    class_fare_avg=dataset['Fare'].groupby(dataset['Pclass']).mean()
    first_class_fare_avg=pd.Series.tolist(class_fare_avg)[0]
    second_class_fare_avg=pd.Series.tolist(class_fare_avg)[1]
    third_class_fare_avg=pd.Series.tolist(class_fare_avg)[2]
    #assegno ai valori mancanti di della colonna dataset['Fare'], le medie del prezzo pagato dalla classe di viaggio di appartenenza
    dataset.loc[(dataset['Pclass'] ==1)&(dataset['Fare']==-1), 'Fare'] = first_class_fare_avg
    dataset.loc[(dataset['Pclass'] ==2)&(dataset['Fare']==-1) , 'Fare'] = second_class_fare_avg
    dataset.loc[(dataset['Pclass'] ==3)&(dataset['Fare']==-1) , 'Fare'] = third_class_fare_avg
    #dataset['Log Fare']=np.log(dataset['Fare']+1) #evito numeri negativi nel dataset anche in caso di normalizzazione
    #Assegno "Southhampton" come porto di partenza dataset['Embarked'] dei passeggeri 61 e 829 del train_set 
    dataset['Embarked']=dataset['Embarked'].fillna('S')
    #Trasformo le informazioni sul sesso dei passeggeri in numeri: assegno 0 alle femmine, e 1 ai maschi
    dataset['Sex'] =dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    #Trasformo in una variabile numerica le informazioni relative al porto di partenza, in base alla distanza da New York
    dataset["Distance to NY"]=dataset['Embarked'].map({'S':5950,'C':5800,'Q':5210})
    #Dal momento che minore è la distanza da New York, minore è stato il prezzo in media pagato dai passeggeri, 
    #divido la 'Fare' per la "Distance to NY", per rendere nullo l'effetto distanza
    dataset["Fare per 1000km"]= 1000*dataset['Fare']/dataset["Distance to NY"]
    # I "Familiars" sono la somma dei coniugi, dei cognati, dei genitori, e dei figli
    dataset['Familiars']=dataset['SibSp']+dataset['Parch']
    # Creo una nuova variabile, identificando le donne o i bambini. Infatti è noto che hanno avuto la precedenza di imbarco nelle scialuppe
    dataset["WomenOrChildren"]=np.zeros(dataset.shape[0])
    #Assegno "1" alle e donne o i bambini, intesi come persone aventi età inferiore ai 18 anni. Assegno "0" ai datapoints restanti, uomini maggiorenni
    dataset.loc[(dataset['Age'] <18) | (dataset['Sex']==0), 'WomenOrChildren'] = 1
    # Assegno l'età media del dataset, alle persone di cui non si conosce l'età
    dataset['Age'][np.isnan(dataset['Age'])] = age_avg
    dataset["WomenOrChildren"]=np.zeros(dataset.shape[0])
    dataset.loc[(dataset['Age'] <18) | (dataset['Sex']==0), 'WomenOrChildren'] = 1
    #converto la colonna dataset['WomenOrChildren'] nel tipo int invece che float
    dataset['WomenOrChildren']=dataset['WomenOrChildren'].astype(int)
    #sposto la colonna dataset['Survived'] in modo da poter raggruppare più facilmente le features
    col_at_end = dataset['Survived']
    dataset.pop('Survived')
    dataset['Survived']=col_at_end
    
#mostro ad esempio il train_df risultato
train_df

Tolgo le colonne che non forniscono, a mio avviso, ulteriori infommazioni trasformabili in numero. Ad esempio il codice della cabina è disponibile solo per la prima classe, ed eventuali diversità tra le cabine di questa classe si riflettono nel prezzo, e quindi probabilmente sulle possibilità di sopravvivenza. La colonna 'Embarked' è già stata convertita in numero con la colonna "Distance to NY".
Creo inoltre un nuovo dataset all_df che combina il train_df e il test_df, ai fini della normalizzazione delle features

In [ ]:
drop_elements = ['Name', 'Ticket', 'Cabin', 'SibSp','Embarked']
train_df = train_df.drop(drop_elements, axis = 1)
test_df=test_df.drop(drop_elements, axis = 1)
all_df=pd.concat([train_df,test_df])
train_df
test_df
all_df

Ora il dataset di train ha tutte le celle piene e tutte convertite in formato numerico. Lo stesso vale per il dataset di test, eccezione fatta ovviamente per la colonna dataset['Survived'] di cui non abbiamo informazioni, e di cui si proverà a prevederne i valori mediante opportuni classificatori e tecniche di regressione

Con la cella del notebook seguente si mostra come inserendo come previsione di sopravvivenza l'informazione se una persona è Donna o Bambino, si riesce ad ottenere tramite Kaggle una già discreta test accuracy (circa 73%). Si utilizzeranno vari classificatori per aumentare la test accuracy, ma intanto la variabile test_df['WomenOrChildren'] appare valida nell'influenzare le chances di sopravvivenza

In [ ]:
WoC=pd.Series(test_df['WomenOrChildren'])

results=pd.DataFrame({'PassengerId':test_df['PassengerId'], 'Survived':WoC})
results.to_csv(r"WomenOrChildren.csv", index=False) #always add r else zeros in csv are not visible
#test accuracy submitted with Kaggle: 0.73205
print('Done!')

Nella cella sottostante si elencano in una lista tutte le variabili utilizzate
Poi chiamo X (per ora) il DataFrame di train di tutte le features utilizzate
Chiamo y la Pandas.Series delle labels, ossia "0" se il passeggero è morto nel naufragio, e "1" se è sopravvissuto
X_te è il dataframe delle features per quanto riguarda i datapoints di test

In [ ]:
all_variables=['Pclass','Sex','Age','Parch','Fare','Distance to NY','Fare per 1000km','Familiars','WomenOrChildren']

X_all_df=all_df[all_variables]
X=train_df[all_variables]
y=train_df['Survived']
X_te=test_df[all_variables]

In [ ]:
X.describe()

**GRAFICI DESCRITTIVI**

In [ ]:
fig, axs = plt.subplots(3, 2, squeeze=False, figsize=(14, 16))
train_df.Age.plot.hist(bins=16, title='Passengers Age  '+'(n=891)', ax=axs[0, 0],rwidth=0.9) 
axs[0,0].set_xlabel("Age")

fare = train_df.Fare
fare.plot.hist(bins=30, title='Fare  '+'(n=891)', ax=axs[0,1],rwidth=1)
axs[0,1].set_xlabel("Fare")

col = ['red', 'green']
train_df.groupby('Survived').agg('count')['Age'].plot.pie(title='Survived  '+'(n=891)', ax=axs[1,0],labels=["Dead","Survived"],colors=col,autopct='%1.1f%%',shadow=True)
axs[1, 0].set_ylabel('')

col = ['lightcoral', 'blue']
train_df.groupby('Sex').agg('count')['Age'].plot.pie(title='Sex  '+'(n=891)', ax=axs[1, 1],labels=["Females","Males"],colors=col,autopct='%1.1f%%',shadow=True)
axs[1, 1].set_ylabel('')

col=["#0066ff","#0099ff","#00ccff"]
train_df.groupby('Pclass').agg('count')['Age'].plot.pie(title='Class  '+'(n=891)', ax=axs[2,0],labels=["1-st class","2-nd class", "3-rd class"],colors=col,autopct='%1.1f%%',shadow=True)
axs[2, 0].set_ylabel('')

col=["#0066ff","lawngreen"]
train_df.groupby('WomenOrChildren').agg('count')['Age'].plot.pie(title='Women or Children  '+'(n=891)', ax=axs[2,1],labels=["Adult Male","Women or Children"],colors=col,autopct='%1.1f%%',shadow=True)
axs[2, 1].set_ylabel('')

plt.subplots_adjust(wspace=0.3, hspace=0.35)

Clono il dataset di train X in W per aggiungere in W delle colonne utili per la descrizione dei dati

In [ ]:
W=X
W['Survived']=y.values
# Uomini e donne sopravvissuti
W['Men survived']=np.zeros(W.shape[0])
W.loc[(W['Survived'] ==1) & (W['Sex']==1), 'Men survived'] = 1
W['Women survived']=np.zeros(W.shape[0])
W.loc[(W['Survived'] ==1) & (W['Sex']==0), 'Women survived'] = 1

total_men=np.round(sum(train_df['Sex']==1))
total_women=np.round(sum(train_df['Sex']==0))
men_survived=np.round(sum(W['Men survived']))
men_dead=total_men-men_survived
women_survived=np.round(sum(W['Women survived']))
women_dead=total_women-women_survived

**Persone sopravvissute per classi di viaggio**

In [ ]:
W['First-Class survived']=np.zeros(W.shape[0])
W['Second-Class survived']=np.zeros(W.shape[0])
W['Third-Class survived']=np.zeros(W.shape[0])
W.loc[(W['Survived'] ==1) & (W['Pclass']==1), 'First-Class survived'] = 1
W.loc[(W['Survived'] ==1) & (W['Pclass']==2), 'Second-Class survived'] = 1
W.loc[(W['Survived'] ==1) & (W['Pclass']==3), 'Third-Class survived'] = 1
total_1st_class=sum(train_df['Pclass']==1)
total_2nd_class=sum(train_df['Pclass']==2)
total_3rd_class=sum(train_df['Pclass']==3)

first_class_survived=np.round(sum(W['First-Class survived']))
first_class_dead=total_1st_class-first_class_survived
second_class_survived=np.round(sum(W['Second-Class survived']))
second_class_dead=total_2nd_class-second_class_survived
third_class_survived=np.round(sum(W['Third-Class survived']))
third_class_dead=total_3rd_class-third_class_survived
third_class_survived

**Adulti e non adulti sopravvissuti**

In [ ]:
W['Non-adult survived']=np.zeros(W.shape[0])
W['Adult survived']=np.zeros(W.shape[0])
W.loc[(W['Survived'] ==1) & (W['Age'] <18), 'Non-adult survived'] = 1
W.loc[(W['Survived'] ==1) & (W['Age']>=18), 'Adult survived'] = 1
total_non_adult=np.round(sum(train_df['Age'] <18))
total_adult=np.round(sum(train_df['Age']>=18))


non_adult_survived=np.round(sum(W['Non-adult survived']))
non_adult_dead=total_non_adult-non_adult_survived
adult_survived=np.round(sum(W['Adult survived']))
adult_dead=total_adult-adult_survived

**"Donne e Bambini" sopravvissuti**

In [ ]:
W['WomenOrChildren survived']=np.zeros(W.shape[0])
W['Adult males survived']=np.zeros(W.shape[0])
W.loc[(W['Survived'] ==1) & (W['WomenOrChildren'] ==1), 'WomenOrChildren survived'] = 1
W.loc[(W['Survived'] ==1) & (W['WomenOrChildren'] ==0), 'Adult males survived'] = 1
total_WomenOrChildren=np.round(sum(train_df['WomenOrChildren']==1))
total_adult_males=np.round(sum(train_df['WomenOrChildren']==0))

WomenOrChildren_survived=sum(W['WomenOrChildren survived']==1)
WomenOrChildren_dead=total_WomenOrChildren-WomenOrChildren_survived
adult_males_survived=np.round(sum(W['Adult males survived']==1))
adult_males_dead=total_adult_males-adult_males_survived

**Costruzione del multiple bar chart**

In [ ]:
#LISTA DEI VALORI DELLE COLONNE DEL MULTIPLE BAR CHART

men_women_survived = [men_survived, women_survived]
men_women_dead = [men_dead, women_dead]

class_survived = [first_class_survived, second_class_survived, third_class_survived]
class_dead = [first_class_dead, second_class_dead, third_class_dead]

adultness_survived=[non_adult_survived,adult_survived]
adultness_dead=[non_adult_dead,adult_dead]

donnebambini_or_not_survived = [WomenOrChildren_survived,adult_males_survived]
donnebambini_or_not_dead = [WomenOrChildren_dead,adult_males_dead]

#FUNZIONE PER VISUALIZZARE I VALORI DELLE COLONNE DEL MULTIPLE BAR CHART SOPRA LE BARRE
# codice riadattatato da https://matplotlib.org/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py
def autolabel(rects,ax): 
    """si aggiunge una label sopra ogni rettangolo, mostrando la sua altezza (che è il valore del rect)"""
    rect_list=[]
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # offset verticale di 3 punti
                    textcoords="offset points",
                    ha='center', va='bottom')

###
fig=plt.figure(figsize=(10,12))

# Uomini e Donne Sopravvissute
labels = ['Men', 'Women']
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

ax = plt.subplot2grid((2,2),(0,0),rowspan=1,colspan=1)
rects1 = ax.bar(x - width/2, men_women_survived, width, label='Survived', color='green')
rects2 = ax.bar(x + width/2, men_women_dead, width, label='Dead', color='red')
ax.set_ylabel('Passengers')
ax.set_title('Number of Men and Women survived')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
autolabel(rects1,ax)
autolabel(rects2,ax)

# Persone Sopravvissute per classi di passeggero
labels = ['1st class', '2nd class','3rd class']
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars
ax1 = plt.subplot2grid((2,2),(0,1),rowspan=1,colspan=1)
rects1 = ax1.bar(x - width/2, class_survived, width, label='Survived', color='green')
rects2 = ax1.bar(x + width/2, class_dead, width, label='Dead', color='red')
ax1.set_ylabel('Passengers')
ax1.set_title('People survived by passenger class')
ax1.set_xticks(x)
ax1.set_xticklabels(labels)
ax1.legend()
autolabel(rects1,ax1)
autolabel(rects2,ax1)

# Bambini e Adulti Sopravvissuti
labels = ['Non-adult', 'Adult']
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

ax2 = plt.subplot2grid((2,2),(1,0),rowspan=1,colspan=1)
rects1 = ax2.bar(x - width/2, adultness_survived, width, label='Survived', color='green')
rects2 = ax2.bar(x + width/2, adultness_dead, width, label='Dead', color='red')
ax2.set_ylabel('Passengers')
ax2.set_title('Number of Non-adults and Adults survived')
ax2.set_xticks(x)
ax2.set_xticklabels(labels)
ax2.legend()
autolabel(rects1,ax2)
autolabel(rects2,ax2)

# Donne&Bambini e AdultiMaschi Sopravvissuti
labels = ['Wom & Non-adult', 'Adult Males']
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

ax3 = plt.subplot2grid((2,2),(1,1),rowspan=1,colspan=1)
rects1 = ax3.bar(x - width/2, donnebambini_or_not_survived, width, label='Survived', color='green')
rects2 = ax3.bar(x + width/2, donnebambini_or_not_dead, width, label='Dead', color='red')
ax3.set_ylabel('Passengers')
ax3.set_title('Women or Children survived')
ax3.set_xticks(x)
ax3.set_xticklabels(labels)
ax3.legend()
autolabel(rects1,ax3)
autolabel(rects2,ax3)

plt.subplots_adjust(wspace=0.3, hspace=0.35)

plt.show()

In [ ]:
pass_class=train_df.groupby("Pclass")
pass_class

**RIDUZIONE DELLA DIMENSIONALITA' ATTRAVERSO LA STEPWISE BACKWARD REGRESSION**

Si è voluto verificare se i modelli forniscono migliori indici di accuracy di convalida, ricorrendo ad una riduzione del numero di caratteristiche.
Si è voluto utilizzare l'approccio della stepwise backward regression, utile per l'eliminazione delle variabili non significative. 
In questo caso la misura della significatività è data dal p-value. Maggiore è il p-value "P>|t|", minore è la statistica T (che è la standardizzazione dei coefficienti di regressione) e minore è la significatività della variabile. Assumendo la normalità delle variabili, e assumendo l'ipotesi nulla che le variabili siano indipendenti dalla Y, il p-value misura la probabilità di ottenere statistiche T uguali o meno probabili di quello osservato durante il test. Dunque, quanto più  T è grande (il coefficiente di regressione è grande rispetto alla sua deviazione standard), tanto minore sarà il p-value, e tanto minore sarà la plausibilità dell'ipotesi nulla di indipendenza
Una tipica soglia massima scelta per il p-value è 0.05, che corrisponde a una statistica T = 2 (in caso di perfetta normalità dei coefficienti di regressione corrisponde a 1.96)
L'algoritmo utilizzato scarta di volta in volta la variabile con il p-value più alto (ad eccezione dell'intercetta), fintanto che rimangono le sole variabili che presentano un p-value inferiore a 0.05. Per fare questo, una volta eliminata una variabile viene ristimato il modello di regressione
Le variabili rimaste sono significative, mentre il coefficiente delle variabili scartate viene considerato pari a 0. Un altro algoritmo di riduzione della dimensionalità è l'Analisi delle Componenti Principali (PCA), in cui dapprima vengono ortogonalizzate le diverse features, che quindi diventano tra loro indipendenti. Poi viene via via scelta la componente principale che è la variabile ortogonalizzata che presenta la massima varianza. Il PCA, tuttavia, non tiene conto della label, e si presta di più a casi di unsupervised learning. La selezione delle variabili basata sulla Stepwise backward regression riguarda direttamente la causalità tra le features e la variabile target, pertanto è un appropriato metodo di selezione delle variabili

In [ ]:
#STEPWISE LINEAR REGRESSION

def stepwise_selection(X, y, 
                       initial_list=['Pclass', 'Sex','Age','Parch','Fare','Distance to NY','Fare per 1000km','Familiars','WomenOrChildren'], 
                       threshold_out = 0.05, 
                       verbose=True):
        
    included = list(initial_list)
    while True:
        changed=False
        

        # backward step
        # si istanzia un modello di regressione di tipo OLS (Ordinary Least Squares)
        model = sm.OLS(y, sm.add_constant(X[included])).fit()
        # si stampano i risultati del modello: coefficienti beta, statistiche T, p-value, R^2, adjusted R^2 ecc. All'inizio vengono prese in considerazione tutte le variabili
        print(model.summary()) 
        # si considerano i p-value di tutte le variabili tranne quello dell'intercetta, che comunque vogliamo mantenere
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # si identifica il massimo p-value (che è legato alla variabile meno significativa)
        if worst_pval > threshold_out: #threshold_out=0.05
            changed=True #il modello cambia. Quindi si deve compiere una nuova regressione
            worst_feature = pvalues.argmax() #si identifica la feature che ha il max p-value
            included.remove(included[worst_feature]) # si rimuove la caratteristica da quelle da includere nel modello di regressione
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed: 
            # se tutte le variabili soddisfano la soglia "threshold_out" non si deve più cambiare il modello e fare una nuova regressione. 
            #Si esce quindi dal ciclo con le sole variabili significative
            break
    return included

result_backward_stepwise_regression = stepwise_selection(X, y)
# Si stampano le sole variabili significative
print("\n\nVariabili significative (p-value<0.05): ",result_backward_stepwise_regression)

# elenco delle variabili significative
variables=result_backward_stepwise_regression







**CONTROLLO SULL'IMPIEGO DI TUTTE LE VARIABILI O SOLO DELLE SIGNIFICATIVE**

Nel primo caso usa: 
full_variables=True

Nel secondo usa:
full_variables=False

In [ ]:
full_variables=True

In [ ]:
if full_variables==True:
    X=train_df[all_variables]   ####### ALL FEATURES X=train_df[all_variables]  ######### FEATURE SELECTION X=train_df[variables] 
else:
    X_te=test_df[variables] ####### ALL FEATURES X_te=test_df[all_variables]  ######### FEATURE SELECTION X_te=test_df[variables]

# Definizione del features_train set e del features_test set
features_train=X.values
features_test=X_te.values

#matrice delle features in cui vengono inclusi i datapoints di tutti i due datasets (train e test). La matrice è utile per la normalizzazione delle features
all_df_features=X_all_df.values

outliers detection

In [ ]:
# Rimozione outliers nei dati secondo il modello IsolationForest
iso = IsolationForest(contamination=0.025, random_state=69)
yhat = iso.fit_predict(train_df)

train_df['Mask']=yhat  #se train_df['Mask']=1 l'esempio non è outlier, altrimenti lo è
train_df = train_df[train_df['Mask']==1]

# Definizione di features_train, SENZA GLI OUTLIERS
#### ALL FEATURES vs SELECTED FEATURES
if full_variables==True:
    features_train=train_df[all_variables] 
else:
    features_train=train_df[variables]
    
labels_train=train_df["Survived"]

**NORMALIZZAZIONE DELLE FEATURES** e rimozione outliers

Normalizzazione delle features, tenendo conto sia del train e del test datasets

In [ ]:
# Istanzio un nuovo oggetto MinMaxScaler capace di normalizzare le caratteristiche



mms=MinMaxScaler()

# Definizione delle FEATURES su cui vogliamo effettuare la NORMALIZZAZIONE

#### ALL FEATURES vs SELECTED FEATURES
if full_variables==True:
    all_features_norm=mms.fit_transform(X_all_df)
else:
    all_features_norm=mms.fit_transform(X_all_df[['Pclass', 'Sex', 'Age', 'Familiars', 'WomenOrChildren']])

features_train_norm=all_features_norm[:891]
features_test_norm=all_features_norm[891:] # le colonne di features_test_norm dipendono dall'"if" precedente
labels_train_norm=y.values

#RIMOZIONE degli outliers sui dati NORMALIZZATI

#Definizione del DATASET in base alle FEATURES SELEZIONATE
#### ALL FEATURES vs SELECTED FEATURES
if full_variables==True:
    train_df_norm=pd.DataFrame(features_train_norm, columns=['Pclass','Sex','Age','Parch','Fare','Distance to NY','Fare per 1000km','Familiars','WomenOrChildren'])
else:
    train_df_norm=pd.DataFrame(features_train_norm, columns=['Pclass', 'Sex', 'Age', 'Familiars', 'WomenOrChildren'])
train_df_norm["Survived"]=labels_train_norm
yhat_norm = iso.fit_predict(train_df_norm)
train_df_norm['Mask']=yhat_norm  #se train_df['Mask']=1 l'esempio non è outlier, altrimenti lo è
train_df_norm = train_df_norm[train_df_norm['Mask']==1]

# DEFINIZIONE delle features_train_norm ossia delle variabili di ADDESTRAMENTO NORMALIZZATE
#### ALL FEATURES vs SELECTED FEATURES
if full_variables==True:
    features_train_norm=train_df_norm[all_variables]
else:
    features_train_norm=train_df_norm[variables]

     
labels_train_norm=train_df_norm["Survived"]

Per esempio mostro la normalizzazione delle features del dataset di addestramento. Per assicurare che a medesimi valori delle features corrispondano medesimi valori normalizzati è stato utilizzato il dataset all_df, sono stati calcolati i valori normalizzati, e poi è stato diviso nelle due parti relative ai dati di addestramento (fino alla riga 891) e ai dati di test (oltre la riga 891).
Le operazioni di normalizzazione sono necessarie nei modelli di classificazione basati sulle distanze tra i datapoints. Se due features hanno campi di variazione differenti dovuti alla scala, la componente delle distanze basata sulle caratteristiche avente un range più ampio risulterà indebitamente sopravvalutata, compromettendo l'affidabilità della classificazione. Tra i modelli impiegati in questo esercizio che sono influenzati dalla distanza troviamo: il Support Vector Machine e il K-Nearest Neighbors

**CREAZIONE FOLD DI CONVALIDA**

Nella cella seguente uso un oggetto Kfold, utile per la convalida incrociata dei classificatori e dei modelli di regressione. In questo caso si divide il train set in 10 parti, delle quali su 9 verrà addestrato un modello, di cui verrà valutata la validità in termini di accuracy sulla parte restante. La stessa operazione verrà compiuta cambiando la parte su cui il modello viene valutato, finché tutti i set di convalida non verranno utilizzati
Si usa shuffle=True per selezionare in modo casuale le 10 parti secondo un fisso random_state, che assicura i medesimi risultati in caso di ripetizione del codice

In [ ]:
kf=KFold(10, shuffle=True, random_state=69) 

#10 sono le k divisioni del dataset; shuffle = True mescolo casualmente

**CLASSIFICATORI**
**SUPPORT VECTOR MACHINE**

L’idea principale che sta alla base di un algoritmo di Support Vector Machine è la massimizzazione del margine tra l'iperpiano di separazione e i punti appartenenti ad una certa classe (label) più vicini a questo iperpiano

In [ ]:
validation_accuracies=[]
best_estimators=[]
#Definisco un dizionario dei parametri sui quali voglio calibrare il classificatore. In questo caso i parametri costituiscono la chiave, 
#i rispettivi valori sono invece le liste che a loro volta contengono i possibili valori su cui voglio addestrare il modello.
parameters = {'kernel':['rbf'],'C':[13500,14000,15000,16000], 'gamma': [0.075,0.08,0.085]}
#istanzio un classificatore di tipo support vector machine
clf=svm.SVC()
#creo un oggetto di tipo GridSearchCV che si concatena al clf definito in precedenza. é a tutti gli effetti un classificatore che viene addestrato su tutte le possibili
#combinazioni di parametri presenti all'interno del dizionario "parameters". 
#Si noti che questa operazione viene ripetuta su tutti i metodi di classificazione e regressione 
clf=GridSearchCV(clf,parameters,refit=True,cv=kf) 
#addestro l'insieme dei classificatori sulle features normalizzate in quanto altrimenti i classificatori considerano le distanze così come sono ignorando i diversi ranges delle variabili
# Prima del fitting è dunque necessario portare le features sulla stessa scala, utilizzando le features normalizzate
clf.fit(features_train_norm,labels_train_norm)
# stampo i parametri del migliore modello in fase di convalida, ossia i parametri del modello che garantiscono la migliore accuratezza media sui 10 KFold.
# questa operazione viene ripetuta su tutti i metodi di classificazione e regressione
print("Miglior classificatore Support Vector Machine ",clf.best_params_)
best_par=clf.best_params_ #recupero i parametri del migliore modello
df_perf_SVM=pd.DataFrame()
mean_accuracies=clf.cv_results_['mean_test_score'] #accuratezza media dei modelli di classificazione usati
parameters_tested=clf.cv_results_['params'] #parametri usati dai singoli modelli di classificazione
type_clf="svm.SVC"
# Voglio costruire un DataFrame per valutare le performances (df_perf_SVM['performances']) dei vari classificatori utilizzati in base a:
#df_perf_SVM['type']: cioé il tipo di classificatore
#df_perf_SVM['parameters']: cioé i parametri usati
#Ottenuti i risultati dei vari classificatori di tutti i tipi si costruirà un unico DataFrame che permetterà di confrontare le performances, e poi successivamente calcolare le previsioni dei classificatori selezionati
#Saranno selezionati i classificatori oltre una certa soglia di accuratezza in fase di convalida e i migliori di ogni tipologia
clf_type_list=[]
for i in range(len(parameters_tested)): #costruisco una lista lunga il numero di classificatori usati (cioé il numero di combinazioni dei parametri) che contiene altrettante stringhe uguali che indicano il tipo di classificatore
    clf_type_list.append(type_clf)
df_perf_SVM['type'] = clf_type_list
df_perf_SVM['parameters']=parameters_tested #l'utilizzo di un DataFrame che contenga la lista (o meglio il dizionario) di parametri usati, permette successivamente di recuperare facilmente i parametri, 
#per poi calcolare le previsioni
df_perf_SVM['performances']=mean_accuracies
df_perf_SVM
#definisco il modello di classificazione con i parametri del miglior modello, 
#così posso "fittarlo" e per infine calcolare il vettore di previsioni (418 * 1) della label, utilizzando le features del dataset di test (in questo caso normalizzate) 
best_clf_SVM=svm.SVC(kernel=best_par['kernel'],C=best_par['C'],gamma=best_par['gamma'])
best_clf_SVM.fit(features_train_norm,labels_train_norm) #fit del classificatore
#fitted_preds=best_clf_SVM.predict(features_train_norm) #previsione dentro il campione (non necessario)
preds_SVM=best_clf_SVM.predict(features_test_norm) #PREVISIONE della label "Survived" basandosi sulle FEATURES del dataset di TEST (in questo caso normalizzate)
#Costruzione di un DataFrame delle previsioni del miglior modello del tipo, e creazione di un file sottoponibile a Kaggle
results=pd.DataFrame({'PassengerId':test_df['PassengerId'], 'Survived':preds_SVM})
results.to_csv(r"C:\Users\Tommaso\Desktop\Titanic_prediction_SVM.csv", index=False)

Nelle successive celle vengono ottenute le performances dei vari tipi di classificatori oltre a Support Vector Machine: Gaussian Naive Bayes,Decision Tree, Random Forest, K-Nearest Neighbor,e Logistic Regression. Siccome il procedimento è analogo al codice commentato dettagliatamente per il classificatore SVM, mi concentrerò maggiormente sugli aspetti statistici

#### Gaussian NB**
Le features"train_survived_ratio" e "train_deaths_ratio" sono le probabilità a priori delle labels (respectively "1" (life) and "0" (death)), P(label=0)=0.616 e P(label=1)=0.383. I valori di questi parametri possono leggermente migliorare la validation accuracy

In [ ]:

train_deaths_ratio=1-np.mean(train_df["Survived"])
train_survived_ratio=np.mean(train_df["Survived"])
parameters = {'priors':[[0.5,0.5],[train_deaths_ratio,train_survived_ratio]],'var_smoothing': [1e-9,1e-8]}
clf=GaussianNB()
# istanziazione di una griglia di classificatori sulla base del dizionario dei parametri (parameters)
clf=GridSearchCV(clf,parameters,refit=True,cv=kf)
clf.fit(features_train,labels_train)
print("Miglior classificatore GaussianNB ",clf.best_params_)
best_par=clf.best_params_
df_perf_NB=pd.DataFrame() 
mean_accuracies=clf.cv_results_['mean_test_score']
parameters_tested=clf.cv_results_['params']
type_clf="GaussianNB"
clf_type_list=[]
for i in range(len(parameters_tested)):
    clf_type_list.append(type_clf)
#Costruzione DataFrame delle performance di accuratezza in fase di validazione
df_perf_NB['type'] = clf_type_list
df_perf_NB['parameters']=parameters_tested
df_perf_NB['performances']=mean_accuracies
#Dichiarazione del miglior classificatore (per accuratezza in fase di validazione)
best_clf_NB=GaussianNB(priors=best_par['priors'],var_smoothing=best_par['var_smoothing'])
best_clf_NB.fit(features_train,labels_train)
#Previsioni dataset di test
preds_NB=best_clf_NB.predict(features_test)
#Costruzione di un DataFrame delle previsioni del miglior modello del tipo, e creazione di un file sottoponibile a Kaggle
results=pd.DataFrame({'PassengerId':test_df['PassengerId'], 'Survived':preds_NB})
results.to_csv(r"C:\Users\Tommaso\Desktop\Titanic_prediction_NB.csv", index=False)

**DECISION TREE**

Questo modello di classificazione è un sistema di suddivisione dei dati di addestramento, prendendo decisioni sulla base di una sequenza di domande. Ogni nodo (che contiene dati) viene ripartito in ulteriori nodi massimizzando l'Information Gain. Per ulteriori dettagli si consulti la relazione

In [ ]:
parameters = {'max_depth':[7,8,10],'min_samples_split': [9,10,11,12],'min_samples_leaf':[1,2,3],'random_state':[69]}
clf=tree.DecisionTreeClassifier(criterion="gini")
# istanziazione di una griglia di classificatori sulla base del dizionario dei parametri (parameters)
clf=GridSearchCV(clf,parameters,refit=True,cv=kf)
clf.fit(features_train,labels_train)
print("Miglior classificatore Decision Tree ",clf.best_params_)
best_par_DT=clf.best_params_
df_perf_DT=pd.DataFrame() 
mean_accuracies=clf.cv_results_['mean_test_score']
parameters_tested=clf.cv_results_['params']
type_clf="tree.DecisionTreeClassifier"
clf_type_list=[]
for i in range(len(parameters_tested)):
    clf_type_list.append(type_clf)
#Costruzione DataFrame delle performance di accuratezza in fase di validazione
df_perf_DT['type'] = clf_type_list
df_perf_DT['parameters']=parameters_tested
df_perf_DT['performances']=mean_accuracies
#Dichiarazione del miglior classificatore (per accuratezza in fase di validazione)
best_clf_DT=tree.DecisionTreeClassifier(max_depth=best_par_DT['max_depth'],min_samples_split=best_par_DT['min_samples_split'],min_samples_leaf=best_par_DT['min_samples_leaf'])
best_clf_DT.fit(features_train,labels_train)

fitted_preds=best_clf_DT.predict(features_train)
#Previsioni dataset di test
preds_DT=best_clf_DT.predict(features_test)
#Costruzione di un DataFrame delle previsioni del miglior modello del tipo, e creazione di un file sottoponibile a Kaggle
results=pd.DataFrame({'PassengerId':test_df['PassengerId'], 'Survived':preds_DT})
results.to_csv(r"C:\Users\Tommaso\Desktop\Titanic_prediction_DT.csv", index=False)

df_perf_DT

Codice utile per la rappresentazione grafica del Decision Tree ottimo

In [ ]:
fig, ax = plt.subplots(figsize=(50, 24))
tree.plot_tree(best_clf_DT, fontsize=6)
plt.savefig('tree_high_dpi', dpi=100)
plt.show()

**RANDOM FOREST with optimized trees**

Il classificatore Random Forest è un esempio di ensemble method nel machine learning. Infatti, una foresta casuale può essere considerata come un insieme (ensemble) di alberi decisionali. In particolare il classificatore Random Forest, attraverso una votazione a maggioranza delle previsioni dei singoli alberi decisionali, calcola un vettore di previsioni. 

In [ ]:
###RANDOM FOREST
# Per gli alberi della foresta si utilizzano i parametri ottimizzati degli alberi decisionali (Decision Tree).
#  Ho riscontrato che utilizzando i parametri ottimizzati si ottiene in fase di validazione un'accuratezza leggermente superiore (2-3% in più)
# n_estimators
parameters = {'n_estimators':[30,40,50,60],'max_features':[2,3,4,5],'n_jobs':[4],'max_depth':[best_par_DT['max_depth']],'min_samples_split':[best_par_DT['min_samples_split']],'min_samples_leaf':[best_par_DT['min_samples_leaf']],'random_state':[69]}
clf=RandomForestClassifier(criterion='gini')
# istanziazione di una griglia di classificatori sulla base del dizionario dei parametri (parameters)
clfRF=GridSearchCV(clf,parameters,refit=True,cv=kf)
clfRF.fit(features_train,labels_train)
print("Miglior classificatore Random Forest ",clfRF.best_params_)
best_par_RF=clfRF.best_params_
df_perf_RF=pd.DataFrame() 
mean_accuracies=clfRF.cv_results_['mean_test_score']
parameters_tested=clfRF.cv_results_['params']
type_clf="RandomForestClassifier"
clf_type_list=[]
for i in range(len(parameters_tested)):
    clf_type_list.append(type_clf)
#Costruzione DataFrame delle performance di accuratezza in fase di validazione
df_perf_RF['type'] = clf_type_list
df_perf_RF['parameters']=parameters_tested
df_perf_RF['performances']=mean_accuracies
#Dichiarazione del miglior classificatore (per accuratezza in fase di validazione)
best_clf_RF=RandomForestClassifier(n_estimators=best_par_RF['n_estimators'],max_features=best_par_RF['max_features'],max_depth=best_par_DT['max_depth'],min_samples_split=best_par_DT['min_samples_split'],min_samples_leaf=best_par_DT['min_samples_leaf'])

best_clf_RF.fit(features_train,labels_train)
fitted_preds=best_clf_RF.predict(features_train)
#Previsioni dataset di test
preds_RF=best_clf_RF.predict(features_test)
#Costruzione di un DataFrame delle previsioni del miglior modello del tipo, e creazione di un file sottoponibile a Kaggle
results=pd.DataFrame({'PassengerId':test_df['PassengerId'], 'Survived':preds_RF})
results.to_csv(r"C:\Users\Tommaso\Desktop\Titanic_prediction_RF.csv", index=False)

df_perf_RF

**K-NEAREST NEIGHBORS**

Questo metodo di classificazione consiste nell’assegnare ad un qualsiasi punto, in uno spazio vettoriale multidimensionale, avente coordinate x_1, x_2,..., x_m, la label prevalente di un certo numero k di train datapoints vicini. 

In [ ]:
parameters={'n_neighbors':[5,15,21,20,25,30,31,35,41],'weights': ['uniform']}
clf=KNeighborsClassifier()
# istanziazione di una griglia di classificatori sulla base del dizionario dei parametri (parameters)
clf=GridSearchCV(clf,parameters,refit=True,cv=kf)
clf.fit(features_train_norm,labels_train_norm)
print("Miglior classificatore KNeighborsClassifier ",clf.best_params_)
best_par=clf.best_params_
df_perf_KNN=pd.DataFrame() 
mean_accuracies=clf.cv_results_['mean_test_score']
parameters_tested=clf.cv_results_['params']
type_clf="KNeighborsClassifier"
clf_type_list=[]
for i in range(len(parameters_tested)):
    clf_type_list.append(type_clf)
#Costruzione DataFrame delle performance di accuratezza in fase di validazione
df_perf_KNN['type'] = clf_type_list
df_perf_KNN['parameters']=parameters_tested
df_perf_KNN['performances']=mean_accuracies
#Dichiarazione del miglior classificatore (per accuratezza in fase di validazione)
best_clf=KNeighborsClassifier(n_neighbors=best_par['n_neighbors'],weights=best_par['weights'])
best_clf.fit(features_train_norm,labels_train_norm)
#Previsioni dataset di test
preds_KNN=best_clf.predict(features_test_norm)
#Costruzione di un DataFrame delle previsioni del miglior modello del tipo, e creazione di un file sottoponibile a Kaggle
results=pd.DataFrame({'PassengerId':test_df['PassengerId'], 'Survived':preds_KNN})
results.to_csv(r"C:\Users\Tommaso\Desktop\Titanic_prediction_KNN.csv", index=False)

df_perf_KNN

**LOGISTIC REGRESSION.**
Viene usata la Penalty L2

Con questo modello di regressione, la cui variabile dipendente è limitata asintoticamente tra 0 e 1, è possibile prevedere valori della label binari ("0" oppure "1")

In [ ]:
parameters={'C':[10,7.5,6,5,4,3,1,0.5],'solver': ['lbfgs'],'max_iter':[200]}
clf = LogisticRegression()
# istanziazione di una griglia di classificatori sulla base del dizionario dei parametri (parameters)
clf=GridSearchCV(clf,parameters,refit=True,cv=kf)
clf.fit(features_train_norm,labels_train_norm)
print("Miglior classificatore LogisticRegression ",clf.best_params_)
best_par=clf.best_params_
df_perf_LogReg=pd.DataFrame() 
mean_accuracies=clf.cv_results_['mean_test_score']
parameters_tested=clf.cv_results_['params']
type_clf="LogisticRegression"
clf_type_list=[]
for i in range(len(parameters_tested)):
    clf_type_list.append(type_clf)
#Costruzione DataFrame delle performance di accuratezza in fase di validazione
df_perf_LogReg['type'] = clf_type_list
df_perf_LogReg['parameters']=parameters_tested
df_perf_LogReg['performances']=mean_accuracies
#Dichiarazione del miglior classificatore (per accuratezza in fase di validazione)
best_clf=LogisticRegression(C=best_par['C'])
best_clf.fit(features_train_norm,labels_train_norm)
acc=accuracy_score(fitted_preds,labels_train_norm)
#Previsioni dataset di test
preds_LogReg=best_clf.predict(features_test_norm)
#Costruzione di un DataFrame delle previsioni del miglior modello del tipo, e creazione di un file sottoponibile a Kaggle
results=pd.DataFrame({'PassengerId':test_df['PassengerId'], 'Survived':preds_LogReg})
results.to_csv(r"C:\Users\Tommaso\Desktop\Titanic_prediction_LogReg.csv", index=False)

df_perf_LogReg

Concateno i DataFrame dei classificatori utilizzati fino a qui, per confrontare le performances dei classificatori

In [ ]:
df_perf_all=pd.concat([df_perf_NB,df_perf_SVM,df_perf_DT,df_perf_RF,df_perf_KNN,df_perf_LogReg])

La lista index_df_perf_all , costruita con il ciclo for contiene un range di numeri interi da 0 fino al numero totale di modelli in df_perf_all.
Questa lista viene inserita come indice, in modo che ogni classificatore abbia un indice unico (attraverso l'istruzione df_perf_all.set_index(df_perf_all['new index']))

In [ ]:
index_df_perf_all=[]
for it in range(df_perf_all.shape[0]):
    index_df_perf_all.append(it)
df_perf_all['new index']=index_df_perf_all
df_perf_all=df_perf_all.set_index(df_perf_all['new index'])
df_perf_all = df_perf_all.drop('new index', 1)

In [ ]:
# Si selezionano i classificatori con un'accuratezza di convalida superiore a una certa soglia
df_perf_eligible=df_perf_all.loc[(df_perf_all['performances'] > 0.834)]
df_perf_eligible

In [ ]:
# Si selezionano i migliori classificatori di ogni tipo
df_perf_group_best=df_perf_all.loc[df_perf_all.groupby('type').performances.idxmax()]
df_perf_group_best

Si uniscono i due DataFrame precedenti, rimuovendo i duplicati. Così si ottengono i classificatori che ho ritenuto idonei a esprimere previsioni
Come esempio di applicazione di un ensemble method, l'intenzione è quella di calcolare una media (ponderata) dei punteggi, tendendo conto di tutti i classificatori idonei.
Così si potrebbe ottenere un meta-classificatore che potrebbe garantire una predizione più accurata e consistente.
L'idea che qui si vuole implementare è simile al majority voting, in cui la predizione più frequente dei classificatori per un datapoint risulterà quella assunta dal meta-classificatore.
Più in dettaglio, però, si è scelto di assegnare diversi pesi ai classificatori votanti, basati sulla loro accuratezza in fase di convalida

In [ ]:
# Si uniscono i dataframes e si rimuovono i duplicati
df_perf_voting=pd.concat([df_perf_eligible,df_perf_group_best])
df_perf_voting = df_perf_voting.reset_index()
df_perf_voting=df_perf_voting.drop_duplicates(subset='new index')
# Versione finale dei classificatori votanti
df_perf_voting

Assegnazione pesi ai classificatori

In [ ]:
# si calcola, come somma totale dei pesi, la somma delle statistiche di accuratezza in fase di convalida
sum_weights=sum(df_perf_voting['performances'])
# il peso "elettorale" di ogni classificatore è dato dal rapporto tra la sua accuratezza e la somma dei pesi. Così facendo,
# si attribuisce una maggiore importanza ai classificatori più performanti 
df_perf_voting['weights']=df_perf_voting['performances']/sum_weights
df_perf_voting=df_perf_voting.sort_values(['type', 'performances'], ascending=[True, False])
index_df_perf_voting=[]
# con le righe seguenti si riaggiorna in modo incrementalel'indice
for it in range(df_perf_voting.shape[0]):
    index_df_perf_voting.append(it)
df_perf_voting['new index']=index_df_perf_voting
df_perf_voting=df_perf_voting.set_index(df_perf_voting['new index'])
df_perf_voting = df_perf_voting.drop('new index', 1)
df_perf_voting

Con la cella seguente si ottengono i vettori di previsione per ognuno dei classificatori votanti

In [ ]:
# Trasformo il DataFrame precedente in un dictionary per poter recuperare il tipo di classificatore, e i parametri di ogni singolo classificatore votante
dict_clf_voting=df_perf_voting.to_dict('index')
# predictions_vectors è la lista che verrà riempita dai vettori di previsione di ogni singolo classificatore (o modello di regressione)
predictions_vectors=[]
# per ogni elemento del dizionario l'obiettivo è ottenere il vettore di previsioni.
# dato che ogni tipo di classificatore si basa su diversi parametri, bisogna prima specificare il tipo di classificatore utilizzato. 
# Per questo all'interno del ciclo for, è stata approntata una "struttura" ad "if"
for item in dict_clf_voting:
    if dict_clf_voting[item]['type']=="GaussianNB": #per esempio se il tipo di classificatore di un certo item è Gaussian Naive Bayes...
        #... si istanzia un classificatore con il valore specifico di item del parametro 'prior' e poi di 'var_smoothing'
        clf=GaussianNB(priors=dict_clf_voting[item]['parameters']['priors'],var_smoothing=dict_clf_voting[item]['parameters']['var_smoothing'])
        clf.fit(features_train,labels_train) # si addestra sul train set questo specifico classificatore
        preds_NB=clf.predict(features_test) # infine si calcolano le previsioni del dataset di test.
        predictions_vectors.append(preds_NB) # Si aggiunge il vettore delle previsioni alla lista "predictions_vectors", che si popola scorrendo il ciclo for di dict_clf_voting
        # si ripetono queste operazioni anche per gli altri tipi di classificatori, prestando attenzione alla necessità di normalizzare le features per gli algoritmi "KNeighborsClassifier"
        # e "svm.SVC" (Support Vector Machine). Infatti, questi si basano sulle distanze tra i datapoints, e pertanto risentono (se non normalizzati o standardizzati) di feature che hanno differenti unità di misura e campo di variazione
    if dict_clf_voting[item]['type']=="KNeighborsClassifier":
        clf=KNeighborsClassifier(n_neighbors=dict_clf_voting[item]['parameters']['n_neighbors'],weights=dict_clf_voting[item]['parameters']['weights'])
        clf.fit(features_train_norm,labels_train_norm)
        preds_KNN=clf.predict(features_test_norm)
        predictions_vectors.append(preds_KNN)
    if dict_clf_voting[item]['type']=="LogisticRegression":
        clf=LogisticRegression(C=dict_clf_voting[item]['parameters']['C'],solver=dict_clf_voting[item]['parameters']['solver'])
        clf.fit(features_train_norm,labels_train_norm)
        preds_KNN=clf.predict(features_test_norm)
        predictions_vectors.append(preds_KNN)
    if dict_clf_voting[item]['type']=="RandomForestClassifier":
        clf=RandomForestClassifier(n_estimators=dict_clf_voting[item]['parameters']['n_estimators'],max_features=dict_clf_voting[item]['parameters']['max_features'],max_depth=dict_clf_voting[item]['parameters']['max_depth'],min_samples_split=dict_clf_voting[item]['parameters']['min_samples_split'],min_samples_leaf=dict_clf_voting[item]['parameters']['min_samples_leaf'])
        clf.fit(features_train,labels_train)
        preds_RF=clf.predict(features_test)
        predictions_vectors.append(preds_RF)
    if dict_clf_voting[item]['type']=="svm.SVC":
        clf=svm.SVC(C=dict_clf_voting[item]['parameters']['C'],gamma=dict_clf_voting[item]['parameters']['gamma'],kernel=dict_clf_voting[item]['parameters']['kernel'])
        clf.fit(features_train_norm,labels_train_norm)
        preds_SVM=clf.predict(features_test_norm)
        predictions_vectors.append(preds_SVM)
    if dict_clf_voting[item]['type']=="tree.DecisionTreeClassifier":
        clf=tree.DecisionTreeClassifier(max_depth=dict_clf_voting[item]['parameters']['max_depth'],min_samples_split=dict_clf_voting[item]['parameters']['min_samples_split'],min_samples_leaf=dict_clf_voting[item]['parameters']['min_samples_leaf'])
        clf.fit(features_train,labels_train)
        preds_DT=clf.predict(features_test)
        predictions_vectors.append(preds_DT)

Si trasforma la lista dei singoli vettori (418 * 1) di previsione (della sopravvivenza delle persone nel dataset di test) in una matrice.
Facendo il prodotto vettoriale tra i pesi e la matrice delle previsioni, si ottiene il vettore ensemble delle previsioni che è una media ponderata delle previsioni dei singoli classificatori. Il vettore risultante si chiama votes_float
Dopodiché, è necessario arrotondare votes_float in un numero intero per ottenere i valori binari "0" e "1". Perché il vettore sia letto correttamente da kaggle infine è necessario convertirlo in maniera esplicita in interi tramite votes=np.round(votes_float,0).astype(int)
votes è il vettore che esprime definitivamente le previsioni sul data set di test

In [ ]:
#trasformazione della lista dei vettori in una matrice
prediction_matrix=np.array(predictions_vectors)
#estrazione dei pesi e trasposizione per il successivo prodotto vettoriale
weights=np.array(df_perf_voting['weights']).T
#prodotto vettoriale tra i pesi e la matrice delle previsioni
votes_float=np.dot(weights,prediction_matrix)
votes_float_df=pd.DataFrame({'PassengerId':test_df['PassengerId'], 'Survived':votes_float})
#arrotondamento del vettore ottenuto e conversione in tipi interi
votes=np.round(votes_float,0).astype(int)
results=pd.DataFrame({'PassengerId':test_df['PassengerId'], 'Survived':votes})
results.to_csv(r"C:\Users\Tommaso\Desktop\Titanic_prediction_Votes.csv", index=False)
print("Elections done")

In [ ]:
votes_float_df

In [ ]:
results